In [1]:
import common as ak
from operator_plan import OperatorPlan
from cost_packet import CostPacket
import numpy as np
import numpy.typing as npt

%load_ext autoreload
%autoreload 2

In [2]:
item_dict = ak.get_item_dict(lang="en_US")
#char_dict = ak.get_char_dict(lang="en_US")
char_dict = ak.get_char_dict(lang="zh_CN")
module_dict = ak.get_module_dict(lang="zh_CN")

In [3]:
n_items = len(item_dict.keys())
n_chars = len(char_dict.keys())
item_names, item_names_rev = ak.get_item_info(item_dict)
#char_names, char_names_rev = ak.get_char_info(char_dict)
char_names, char_names_rev = ak.get_char_translations(char_dict)

In [4]:
packet_all, _ = ak.get_all_char_all_costs(char_dict, module_dict, n_chars)


In [5]:
plan_nearl = OperatorPlan(
    name = "Passenger",
    elite_range = (0, 2),
    skill_range = (1, 7),
    mastery_range = [
        (0, 0),
        (0, 0),
        (0, 3)
    ],
    module_range = [
        (0, 0),
        (0, 3)
    ]
)

mats_combined = plan_nearl.get_cost(char_names_rev, packet_all)

plan_nearl.pretty_print(item_names, mats_combined)

Passenger:
	E0 -> E2
	SL1 -> SL7
	S3M0 -> S3M3
	M2L0 -> M2L3
Cost:
	Orirock Cube: 10
	Orirock Cluster: 8
	Orirock Concentration: 10
	Ester: 6
	Oriron Shard: 4
	Oriron: 12
	Oriron Cluster: 4
	Oriron Block: 5
	Polyketon: 4
	Polymerization Preparation: 3
	Bipolar Nanoflake: 8
	Crystalline Electronic Unit: 8
	Coagulating Gel: 7
	Polymerized Gel: 4
	Incandescent Alloy Block: 4
	Crystalline Circuit: 4
	Caster Chip: 5
	Caster Dualchip: 4
	Skill Summary - 1: 10
	Skill Summary - 2: 24
	Skill Summary - 3: 43
	LMD: 300000
	Module Data Block: 12
	mod_update_token_1: 60
	mod_update_token_2: 20


# Testing - Loading Gamepress from JSON

In [6]:
import json
with open("gamepress.json") as f:
    gpdata = json.load(f)["operators"]

In [7]:
n_gp_operators = len(gpdata)
gp_plans = np.empty(n_gp_operators, dtype=OperatorPlan)

In [8]:
for i, op in enumerate(gpdata):
    try:
        ## gamepress id doesnt match in-game id data
        name = char_names_rev[op["operator"]]
    except:
        name = op["operator"]
    print(name)
    gp_plans[i] = OperatorPlan(
            name = name,
            elite_range = (op["start_promotion"], op["end_promotion"]),
            skill_range = (op["start_skill"], op["end_skill"]),
            mastery_range = np.transpose((op["start_specs"], op["end_specs"]))
        )

9241
19141
13296
1241
21761
13281
1291
14701
16026
37136
37146
37176
35311
39001
25811
7211
23291
26601
30201
1296
19641
30156
1336
1376
16031
1306
24826
40436
40446
41791
26616
1311
11966
16036
1186
26536
42456
18066
43586
44496
44491
44501
14706
46546
42461
40451
43581


# Test - Serialising OperatorPlan objects

In [9]:
obj_str = plan_nearl.to_json()

new_dict = json.loads(obj_str)
new_plan = OperatorPlan(**new_dict)

print(new_plan)

# Test - User interface

In [20]:
preset_e1 = OperatorPlan(elite_range=(0,1))
preset_e1_SL7 = OperatorPlan(elite_range=(0,1), skill_range=(1,7))
preset_e2 = OperatorPlan(elite_range=(0,2))
preset_e2_SL7 = OperatorPlan(elite_range=(0,2), skill_range=(1,7))
preset_e2_S1M3 = OperatorPlan(elite_range=(0,2), skill_range=(1,7), mastery_range=[(0,3),(0,0),(0,0)])
preset_e2_S2M3 = OperatorPlan(elite_range=(0,2), skill_range=(1,7), mastery_range=[(0,0),(0,3),(0,0)])
preset_e2_S3M3 = OperatorPlan(elite_range=(0,2), skill_range=(1,7), mastery_range=[(0,0),(0,0),(0,3)])
preset_S1M3 = OperatorPlan(mastery_range=[(0,3),(0,0),(0,0)])
preset_S1M3 = OperatorPlan(mastery_range=[(0,0),(0,3),(0,0)])
preset_S1M3 = OperatorPlan(mastery_range=[(0,0),(0,0),(0,3)])
preset_S2M3_S3M3 = OperatorPlan(mastery_range=[(0,0),(0,3),(0,3)])
preset_M9 = OperatorPlan(mastery_range=[(0,3),(0,3),(0,3)])

In [21]:
suzuran = preset_e2_SL7.as_operator("Passenger")
suzuran_total = suzuran.get_cost(char_names_rev, packet_all)
suzuran.pretty_print(item_names, suzuran_total)

Passenger:
	E0 -> E2
	SL1 -> SL7
Cost:
	Orirock Cube: 10
	Orirock Cluster: 8
	Ester: 6
	Oriron Shard: 4
	Oriron: 12
	Oriron Cluster: 4
	Oriron Block: 5
	Polyketon: 4
	Bipolar Nanoflake: 4
	Coagulating Gel: 4
	Caster Chip: 5
	Caster Dualchip: 4
	Skill Summary - 1: 10
	Skill Summary - 2: 24
	Skill Summary - 3: 8
